# Recommender systems

https://cseweb.ucsd.edu/~jmcauley/datasets.html

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
import warnings
warnings.filterwarnings("ignore")
from scipy.sparse import csr_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split
from sklearn.model_selection import train_test_split as tts
from collections import defaultdict
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import gc
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df = pd.read_json('Amazon_Fashion.jsonl', lines=True)

In [3]:
df.shape

(2500939, 10)

In [3]:
df_meta = pd.read_json('meta_Amazon_Fashion.jsonl', lines=True)

In [10]:
df_meta.shape

(826108, 14)

In [11]:
df.head(3)

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5,Pretty locket,I think this locket is really pretty. The insi...,[],B00LOPVX74,B00LOPVX74,AGBFYI2DDIKXC5Y4FARTYDTQBMFQ,2020-01-09 00:06:34.489,3,True
1,5,A,Great,[],B07B4JXK8D,B07B4JXK8D,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,2020-12-20 01:04:06.701,0,True
2,2,Two Stars,One of the stones fell out within the first 2 ...,[],B007ZSEQ4Q,B007ZSEQ4Q,AHITBJSS7KYUBVZPX7M2WJCOIVKQ,2015-05-23 01:33:48.000,3,True


In [13]:
df_meta.head(3).T

,0,1,2
main_category,AMAZON FASHION,AMAZON FASHION,AMAZON FASHION
title,YUEDGE 5 Pairs Men's Moisture Control Cushione...,DouBCQ Women's Palazzo Lounge Wide Leg Casual ...,Pastel by Vivienne Honey Vanilla Girls' Trapez...
average_rating,4.6,4.1,4.3
rating_number,16,7,11
features,[],"[Drawstring closure, Machine Wash]","[Zipper closure, Hand Wash Only]"
description,[],[],[]
price,NaN,NaN,NaN
images,[{'thumb': 'https://m.media-amazon.com/images/...,[{'thumb': 'https://m.media-amazon.com/images/...,[{'thumb': 'https://m.media-amazon.com/images/...
videos,[],[],[]
store,GiveGift,DouBCQ,Pastel by Vivienne


In [4]:
df_filtered = df.groupby('user_id').filter(lambda x: len(x) >= 3)
df_filtered = df_filtered.groupby('parent_asin').filter(lambda x: len(x) >= 3)
#I decided to use only data with users with at least three reviews

In [5]:
df_ratings = df_filtered[['user_id', 'parent_asin', 'rating']]
df_ratings['interaction'] = (df_ratings['rating'] >= 4).astype(int)
user_enc = LabelEncoder()
item_enc = LabelEncoder()
df_ratings['user_idx'] = user_enc.fit_transform(df_ratings['user_id'])
df_ratings['item_idx'] = item_enc.fit_transform(df_ratings['parent_asin'])

In [23]:
df_ratings.shape

(118149, 6)

### Collaborative filtering

In [24]:
ratings_matrix = csr_matrix((df_ratings['rating'], (df_ratings['user_idx'], df_ratings['item_idx'])),
    shape=(df_ratings['user_idx'].nunique(), df_ratings['item_idx'].nunique()))

In [25]:
def recommend_similar_items(item_idx, top_n=5):
    sim_scores = item_similarity[item_idx]
    top_indices = np.argsort(sim_scores)[::-1][1:top_n+1]
    return top_indices, sim_scores[top_indices]

item_similarity = cosine_similarity(ratings_matrix.T)

In [26]:
similar_items, scores = recommend_similar_items(1)
for idx, score in zip(similar_items, scores):
    print(f"Item {item_enc.inverse_transform([idx])[0]} — similarity: {score:.2f}")

Item B01833A8F2 — similarity: 0.32
Item B0178X5E0W — similarity: 0.27
Item B09H6MXJ71 — similarity: 0.05
Item B0CF5FKGV2 — similarity: 0.00
Item B01D23C72A — similarity: 0.00


In [27]:
density = ratings_matrix.count_nonzero() / (ratings_matrix.shape[0] * ratings_matrix.shape[1])
print(f"Matrix density: {density:.6f}")

Matrix density: 0.000089


The user–item interaction matrix exhibits extreme sparsity, with a density of just 0.002% (initially 0.0001% if not filtered), meaning the vast majority of possible user–item pairs have no recorded interactions. This level of sparsity severely limits the effectiveness of traditional collaborative filtering approaches such as user–user and item–item cosine similarity, as there is minimal overlap in the items rated by different users or the users who rated the same items. The sparsity reflects the nature of large e‑commerce datasets, where individual users typically interact with only a tiny fraction of the available catalog. 

### Matrix factorization and SVD model

In [8]:
del ratings_matrix, df_ratings
import gc
gc.collect()

0

In [36]:
ratings_df = df[['user_id', 'parent_asin', 'rating']]
reader = Reader(rating_scale=(ratings_df['rating'].min(), ratings_df['rating'].max()))
data = Dataset.load_from_df(ratings_df, reader)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [37]:
algo = SVD(n_factors=50, biased=True, random_state=42)
algo.fit(trainset)

In [40]:
predictions = algo.test(testset)
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

RMSE: 1.3719
MAE:  1.1213


In [38]:
def get_top_n(predictions, n=10):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

def precision_at_k(predictions, k=10, threshold=4.0):
    user_est_true = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))
    precisions = []
    for uid, user_ratings in user_est_true.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        top_k = user_ratings[:k]
        n_rel = sum((true_r >= threshold) for (_, true_r) in top_k)
        precisions.append(n_rel / k)
    return sum(precisions) / len(precisions)

In [41]:
top_n = get_top_n(predictions)
user = list(top_n.keys())[0]
print(f"Top recommendations for {user}:")
for iid, rating in top_n[user]:
    print(f"Item: {iid}, Predicted rating: {rating:.2f}")

Top recommendations for AEDWOML5U2GCOVCC5VMJ4O4NLRHA:
Item: B083LXCFT6, Predicted rating: 4.38


In [42]:
p_at_10 = precision_at_k(predictions, k=10, threshold=4.0)
print(f"Precision@10: {p_at_10:.4f}")

Precision@10: 0.0754


The SVD model's predicted ratings were reasonably close to actual ratings, but this doesn't tell us how good the recommendations are. To test recommendation quality, we measured Precision@10, which shows how many of the top 10 recommended items users actually liked (rated 4 or higher). The model achieved a Precision@10 of 7.5%. These results show that while SVD can learn some user preferences, the sparse data limits how well it works.

In [22]:
del data, reader, trainset, testset
gc.collect()

0

### Neural Models and Advanced Recommenders

In [30]:
# lightfm, implicit - have some problems with MacOS installation

In [43]:
train_df, val_df = tts(df_ratings, test_size=0.2, random_state=42)
train_users = torch.LongTensor(train_df['user_idx'].values)
train_items = torch.LongTensor(train_df['item_idx'].values)
train_ratings = torch.FloatTensor(train_df['rating'].values)

val_users = torch.LongTensor(val_df['user_idx'].values)
val_items = torch.LongTensor(val_df['item_idx'].values)
val_ratings = torch.FloatTensor(val_df['rating'].values)

train_ds = TensorDataset(train_users, train_items, train_ratings)
val_ds = TensorDataset(val_users, val_items, val_ratings)

train_dl = DataLoader(train_ds, batch_size=1024, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=1024)

In [44]:
class NCF(nn.Module):
    def __init__(self, n_users, n_items, n_factors=64):
        super().__init__()
        self.user_emb = nn.Embedding(n_users, n_factors)
        self.item_emb = nn.Embedding(n_items, n_factors)
        self.fc1 = nn.Linear(n_factors * 2, 128)
        self.fc2 = nn.Linear(128, 64)
        self.out = nn.Linear(64, 1)

    def forward(self, user, item):
        u = self.user_emb(user)
        i = self.item_emb(item)
        x = torch.cat([u, i], dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.out(x).squeeze()

In [45]:
model = NCF(n_users=df_ratings['user_idx'].nunique(), n_items=df_ratings['item_idx'].nunique())

In [46]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

for epoch in range(5):
    model.train()
    train_loss = 0
    for u, i, r in tqdm(train_dl, desc=f"Epoch {epoch+1}"):
        pred = model(u, i)
        loss = loss_fn(pred, r)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_dl)

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for u, i, r in val_dl:
            val_loss += loss_fn(model(u, i), r).item()
    val_loss /= len(val_dl)

    print(f"Epoch {epoch+1}: train_loss={train_loss:.4f} val_loss={val_loss:.4f}")

Epoch 1: 100%|████████████████████████████████| 257/257 [00:05<00:00, 43.32it/s]


Epoch 1: train_loss=2.8181 val_loss=1.8145


Epoch 2: 100%|████████████████████████████████| 257/257 [00:05<00:00, 43.59it/s]


Epoch 2: train_loss=1.6993 val_loss=1.7816


Epoch 3: 100%|████████████████████████████████| 257/257 [00:09<00:00, 27.54it/s]


Epoch 3: train_loss=1.5767 val_loss=1.7668


Epoch 4: 100%|████████████████████████████████| 257/257 [00:06<00:00, 41.61it/s]


Epoch 4: train_loss=1.4618 val_loss=1.7506


Epoch 5: 100%|████████████████████████████████| 257/257 [00:06<00:00, 40.40it/s]


Epoch 5: train_loss=1.3344 val_loss=1.7512


Overfitted training set — training loss decreased to 0.0898, but validation loss increased to 2.92, signaling overfitting. => Add dropout layers (to reduce co-adaptation of neurons), add L2 regularization (to penalize large weights), stop when validation loss stops improving.

In [47]:
class NCF(nn.Module):
    def __init__(self, n_users, n_items, n_factors=64, dropout=0.3):
        super().__init__()
        self.user_emb = nn.Embedding(n_users, n_factors)
        self.item_emb = nn.Embedding(n_items, n_factors)
        
        self.fc1 = nn.Linear(n_factors * 2, 128)
        self.dropout1 = nn.Dropout(dropout)
        
        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(dropout)
        
        self.out = nn.Linear(64, 1)

    def forward(self, user, item):
        u = self.user_emb(user)
        i = self.item_emb(item)
        x = torch.cat([u, i], dim=1)
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        return self.out(x).squeeze()

In [48]:
model = NCF(n_users=df_ratings['user_idx'].nunique(), n_items=df_ratings['item_idx'].nunique())

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
loss_fn = nn.MSELoss()

best_val_loss = np.inf
patience = 3
epochs_no_improve = 0
n_epochs = 30

for epoch in range(n_epochs):
    model.train()
    train_loss = 0
    for u, i, r in train_dl:
        pred = model(u, i)
        loss = loss_fn(pred, r)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_dl)

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for u, i, r in val_dl:
            val_loss += loss_fn(model(u, i), r).item()
    val_loss /= len(val_dl)

    print(f"Epoch {epoch+1}: train_loss={train_loss:.4f}, val_loss={val_loss:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print("Early stopping triggered.")
            break

Epoch 1: train_loss=3.1800, val_loss=1.8540
Epoch 2: train_loss=2.1746, val_loss=1.8682
Epoch 3: train_loss=2.1170, val_loss=1.8551
Epoch 4: train_loss=2.0679, val_loss=1.8043
Epoch 5: train_loss=2.0138, val_loss=1.7663
Epoch 6: train_loss=1.9559, val_loss=1.7403
Epoch 7: train_loss=1.8661, val_loss=1.7191
Epoch 8: train_loss=1.6825, val_loss=1.6922
Epoch 9: train_loss=1.3471, val_loss=1.7246
Epoch 10: train_loss=0.9751, val_loss=1.7933
Epoch 11: train_loss=0.7301, val_loss=1.8863
Early stopping triggered.


In [49]:
def get_top_n_nn_for_user(model, user_id_str, df, user_enc, item_enc, n=10):
    model.eval()

    user_idx = user_enc.transform([user_id_str])[0]
    all_item_indices = np.arange(df['item_idx'].nunique())
    item_tensor = torch.LongTensor(all_item_indices)
    user_tensor = torch.LongTensor([user_idx] * len(all_item_indices))

    with torch.no_grad():
        preds = model(user_tensor, item_tensor).numpy()

    top_n_items = np.argsort(preds)[-n:][::-1]
    recommended_asins = item_enc.inverse_transform(top_n_items)

    return recommended_asins

In [50]:
test_user = df_ratings['user_id'].iloc[0]
top_items = get_top_n_nn_for_user(model, test_user, df_ratings, user_enc, item_enc, n=10)
print(f"Top 10 recommended items for user {test_user}:")
print(top_items)

Top 10 recommended items for user AHREXOGQPZDA6354MHH4ETSF3MCQ:
['B01CZ2F898' 'B01AX8N7OO' 'B01MTMQXHX' 'B01MF8O21E' 'B01E8XTQHW'
 'B00C6FCJBU' 'B007CL42QS' 'B08GKJ7SCQ' 'B01CX8QBJU' 'B00KNHVFN6']


In [51]:
def precision_at_k_ncf(model, val_df, k=10, threshold=4.0):
    model.eval()
    user_est_true = defaultdict(list)

    with torch.no_grad():
        for u, i, r in val_dl:
            preds = model(u, i)
            for uid, iid, pred, true in zip(u, i, preds, r):
                user_est_true[uid.item()].append((pred.item(), true.item()))

    precisions = []

    for uid, user_ratings in user_est_true.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        top_k = user_ratings[:k]
        n_rel = sum(true >= threshold for _, true in top_k)
        precisions.append(n_rel / k)

    return sum(precisions) / len(precisions)

In [52]:
p_at_10 = precision_at_k_ncf(model, val_df, k=10, threshold=4.0)
print(f"Precision@10: {p_at_10:.4f}")

Precision@10: 0.1057


In [53]:
user_est_true = defaultdict(list)
for u, i, r in val_dl:
    preds = model(u, i)
    for uid, iid, pred, true in zip(u, i, preds, r):
        user_est_true[uid.item()].append((pred.item(), true.item()))

for uid, user_ratings in user_est_true.items():
    print(f"user {uid}, est/true: {user_ratings[:3]}")
    break

user 20575, est/true: [(4.676995754241943, 5.0), (3.7491989135742188, 3.0), (3.7491989135742188, 3.0)]


### Hybrid Recommender with SVD + Metadata (TF-IDF + Average Rating)

In [27]:
df_meta_sub = df_meta[['parent_asin', 'title', 'average_rating']].dropna().drop_duplicates('parent_asin')

sampled_asins = df_meta_sub['parent_asin'].drop_duplicates().sample(5000, random_state=42)
df_meta_sub_small = df_meta_sub[df_meta_sub['parent_asin'].isin(sampled_asins)]
df_ratings_small = df_ratings[df_ratings['parent_asin'].isin(sampled_asins)]

df_full = df_ratings_small.merge(df_meta_sub_small, on='parent_asin')

vectorizer = TfidfVectorizer(max_features=1000)
tfidf_matrix = vectorizer.fit_transform(df_meta_sub_small['title'])

In [7]:
del df, df_meta
gc.collect()

0

In [15]:
tfidf_matrix.shape

(5000, 1000)

In [28]:
cosine_sim_matrix = cosine_similarity(tfidf_matrix)

asin_to_idx = {asin: idx for idx, asin in enumerate(df_meta_sub_small['parent_asin'])}
idx_to_asin = {idx: asin for asin, idx in asin_to_idx.items()}
asin_to_title = dict(zip(df_meta_sub_small['parent_asin'], df_meta_sub_small['title']))

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_ratings[['user_id', 'parent_asin', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

svd = SVD()
svd.fit(trainset)

In [29]:
def hybrid_score_fast(user_id, candidate_asins, top_k=10, alpha=0.7, beta=0.2, debug=False):
    scores = []
    user_items = df_ratings[df_ratings['user_id'] == user_id]['parent_asin'].unique()
    seen_idxs = [asin_to_idx[a] for a in user_items if a in asin_to_idx]

    for asin in candidate_asins:
        if asin not in asin_to_idx:
            continue

        asin_idx = asin_to_idx[asin]

        try:
            cf_score = svd.predict(user_id, asin).est
            cf_score = (cf_score - 1) / 4
        except:
            cf_score = 0

        content_sims = cosine_sim_matrix[asin_idx, seen_idxs] if seen_idxs else []
        content_score = np.mean(content_sims) if len(content_sims) else 0

        rating_score = df_meta_sub_small[df_meta_sub_small['parent_asin'] == asin]['average_rating'].values
        rating_score = (rating_score[0] - 1) / 4 if len(rating_score) else 0

        total = alpha * cf_score + beta * content_score + (1 - alpha - beta) * rating_score
        scores.append((asin, total))

        if debug:
            print(f"[{asin}] CF: {cf_score:.2f}, Content: {content_score:.2f}, Rating: {rating_score:.2f} → Total: {total:.2f}")

    scores.sort(key=lambda x: x[1], reverse=True)
    return scores[:top_k]

In [30]:
df_ratings_small.head()

,user_id,parent_asin,rating,interaction,user_idx,item_idx
726,AEICZD35OTDYVWA2KPYWN4PDZ2SA,B08BFQ7ZH8,5,1,6549,18753
1179,AGMZO7NPGHYPLCOCLMQEDGKNCHVA,B012318KE6,5,1,37616,4433
3009,AG5FGCIM6PBJX3JNDRKQMITZTIMA,B08BFQ7ZH8,5,1,30544,18753
3057,AEZP6Z2C5AVQDZAJECQYZWQRNG3Q,B07SN27NK5,4,1,14356,17195
3798,AFTZWAK3ZHAPCNSOT5GCKQDECBTQ,B09PHG13QP,5,1,26306,20272


In [31]:
user_id = 'AEICZD35OTDYVWA2KPYWN4PDZ2SA'#df_ratings['user_id'].iloc[726]
candidate_asins = df_meta_sub_small['parent_asin'].unique()

recommendations = hybrid_score_fast(user_id, candidate_asins, top_k=10, alpha=0.6, beta=0.3)

print(f"Top 10 recommendations for user {user_id}:\n")
for asin, score in recommendations:
    print(f"{asin_to_title[asin]} (ASIN: {asin}) — Score: {score:.2f}")

Top 10 recommendations for user AEICZD35OTDYVWA2KPYWN4PDZ2SA:

QBSM Women's Shawl Wrap Poncho Ruana Capes Open Front Cardigan Blanket Wraps for Fall and Winter (ASIN: B08BFQ7ZH8) — Score: 0.92
Ourlove Fashion Women‘s Knitted Open Poncho Cape Ladies Christmas Shawl/Blanket Long Wrap with Tassel (Black+Red) (ASIN: B01MQGC1H3) — Score: 0.69
JJ Perfection Women's Asymmetric Drape Open Front Long Cardigan Oatmeal M (ASIN: B01M6BTXYP) — Score: 0.69
GUESS Factory Women's Athena Toggle Poncho (ASIN: B01M66LQST) — Score: 0.68
Kumer Women Speckled Fringe Cardigan Fall Knited Tassels Sweater Aztec Stripes Shawl Loose Slash Sweater Poncho (ASIN: B0773MTHSM) — Score: 0.67
Shawl Collar Grandfather Cardigan Oatmeal (ASIN: B00D4ANELK) — Score: 0.67
Women Winter Knitted Cashmere Poncho Capes Shawl Cardigans Sweater Coat (one size, Beige) (ASIN: B01M0LZWDT) — Score: 0.67
Triple9shop Women's Button-Side Front Pocket Poncho Cardigan (Coffee) (ASIN: B015QEEXP2) — Score: 0.66
Women’s Flowy Boho Floral Print

In [34]:
test_asin = df_meta_sub_small['parent_asin'].iloc[0]
test_idx = asin_to_idx[test_asin]

sim_scores = cosine_sim_matrix[test_idx]
top_sim_idxs = sim_scores.argsort()[::-1][1:6]

print(f"Top 5 similar products to: {df_meta_sub_small['title'].iloc[test_idx]}")
for i in top_sim_idxs:
    print(f"{df_meta_sub_small['title'].iloc[i]} (score: {sim_scores[i]:.2f})")

Top 5 similar products to: Women 2pc Gold belts Silver Skinny Belt Elastic Chain Wedding Belt Mirror Metal Waistband
Women’s Girls Skinny Faux Leather Belt Slim Waist Belt Waistband with Metal Buckle for Dress Pants (black, waist below 40") (score: 0.50)
Women's Wide Hook Stretch Elastic Waist Belt Waistband XL CL8961-1 (score: 0.49)
HDE Men's Belt Elastic Stretch Waistband Woven Design Leather Tip Silver Buckle (score: 0.45)
Bibest Magnetic Belts for Boys Grils Toddlers Adjustable Elastic Belt, Pack of 3 (score: 0.42)
Women's Elastic Costume Waist Belt Lace-up Tied Corset Belts for Women Black XXL (score: 0.41)


# SUMMARY

Traditional CF struggles to scale to massive, sparse e-commerce datasets.  
Surprise SVD on user-item matrix: Precision@10: 7.5% - Learns latent patterns; better than raw CF but still affected by data sparsity => Use for registered users with sufficient history (boost repeat engagement and basket size).  
PyTorch-based deep model with embedding layers and dropout regularization: Precision@10: 10.6%, slow in train, needs GPU/optimization => Can power personalized homepage, email campaigns, and retargeting.  
Hybrid Recommender (SVD + TF-IDF + Average Rating): Top Recommendation Score: ~0.92 => Use as default model for guest users, cold-start users, or sparsely rated products.

I acknowledge that my evaluation pipeline wasn't fully consistent across models. Ideally, all models should have used the same filtered dataset and evaluation methods for direct comparison. However, due to hardware limitations like memory overload and kernel crashes, I had to train each model separately. This meant re-initializing from scratch each time, losing the ability to hold all data in memory simultaneously, and sometimes re-generating filtered samples, which introduced variance.
I should have added popularity-based recommendations as a baseline and used consistent metrics across all models, including not just precision but also recall, coverage, diversity, and NDCG for more comprehensive evaluation.